Need to generate data for entities first, then for relationships using the entities 

entites:
- warehouse
- store (many physical, one online) +
- products +
- orders
- customer +
- payment type +

relationships:
- order payment (o - pt)
- customer order (o - c)
- stores orders (o - s)
- items purchased (o - p)
- supplies (w -s)
- warehouse inventory (w - p)
- store inventory (s - p)

In [1]:
# Importing required libraries
import mysql.connector
import pandas as pd
from faker import Faker 
import random
import datetime
import os
from dotenv import load_dotenv

load_dotenv()

In [2]:
# Creating an instance of the faker class, which is used to generate fake, realistic data
fake = Faker()

entity: physical stores

In [3]:
# Creating a function that generates data for brick and mortar store attributes 
def generatePhysicalStore(store_id):

    return {
        "store_id": store_id,
        "phone_number": f"{random.randint(100, 999)}.{random.randint(100, 999)}.{random.randint(100, 999)}",
        "address": f"{fake.building_number()} {fake.street_name()}, Apt {random.randint(111, 999)}",
        "zip_code": fake.zipcode(),
        "city": fake.city(),
        "state": fake.state_abbr(),
        "country": "US",
        # Generating the time in HH:MM:SS format to satisfy the time data type that we used when creating the table
        "weekday_open_time": f"{random.randint(7, 12):02d}:00:00",
        "weekday_close_time": f"{random.randint(14, 23):02d}:00:00",
        "weekend_open_time": f"{random.randint(8, 12):02d}:00:00",
        "weekend_close_time": f"{random.randint(14, 23):02d}:00:00",
    }

entity: warehouse

In [4]:
# Creating a function to generate data for the warehouse attributes
def generateWarehouse(warehouse_id):

    return {
        "warehouse_id": warehouse_id,
        "zip_code": fake.zipcode(),
        "address": f"{fake.building_number()} {fake.street_name()}, Apt {random.randint(111, 999)}",
        "city": fake.city(),
        "state": fake.state_abbr(),
        "country": "US"
}   

entity: order

In [5]:
# Creating a function to generate data for the order attributes
def generateOrder(order_id):

    return {
        "order_id": order_id,
        "order_date": fake.date_between(start_date="-2y", end_date="+1w"),
        "tracking_num": random.randint(111111111111, 999999999999)
}  

entity: customer member

In [6]:
# Creating a function to generate data for the attributes when a customer is a member
def generateCustomerMember(customer_id, member_id):

    fname = fake.first_name()
    lname = fake.last_name()

    return {
        "customer_id": customer_id,
        "member_id": member_id,
        "firstname": fname,
        "lastname": lname,
        "phone_num": f"{random.randint(100, 999)}.{random.randint(100, 999)}.{random.randint(100, 999)}",
        "email": f"{fname}{lname}@gmail.com",
        "rewards_password": random.randint(0000000000, 9999999999),
        "zip_code": fake.zipcode(),
        "address": f"{fake.building_number()} {fake.street_name()}, Apt {random.randint(111, 999)}",
        "city": fake.city(),
        "state": fake.state_abbr(),
        "country": "US"
}   

entity: customer non-member online

In [7]:
# Creating a function to generate data for the attributes when a customer is not a member and purchased online
def generateCustomerNonMembOnline(customer_id):


    fname = fake.first_name()
    lname = fake.last_name()
    
    return {
        "customer_id": customer_id,
        "firstname": fname,
        "lastname": lname,
        "phone_num": f"{random.randint(100, 999)}.{random.randint(100, 999)}.{random.randint(100, 999)}",
        "email": f"{fname}{lname}@gmail.com",
        "address": f"{fake.building_number()} {fake.street_name()}, Apt {random.randint(111, 999)}",
        "zip_code": fake.zipcode(),
        "city": fake.city(),
        "state": fake.state_abbr(),
        "country": "US"
}   

entity: customer non-member in-person

In [8]:
# Creating a function to generate data for the customer entity
def generateCustomerNonMember(customer_id):

    return {
        "customer_id": customer_id
}   

entity: credit card info

In [9]:
# Creating a function to generate data for the customers credit card information
def generateCardInfo(payment_id):
    
    month = random.randint(1, 12)
    year = random.randint(2024, 2034)  # WhAt should range be??? They usually expire within 3-5 years google says
    day = 1

    return {
        "payment_id": payment_id,
        "name_on_card": fake.name(),
        "credit_card_num": random.randint(1111111111111111, 9999999999999999),
        "cvv": random.randint(111, 999),
        "expiration_date": f"{year}-{month:02d}-{day:02d}" ,
}

entity: gift card info 

In [10]:
# Creating a function to generate data for the customers gift card information
def generateGiftCardInfo(payment_id):

    return {
        "payment_id": payment_id,
        "gift_card_num": random.randint(1111111111111111, 9999999999999999),
        "cvv": random.randint(111, 999),
        "balance": random.randint(0, 500)
} 

MAIN FUNCTION: literally generating the data

create and open file for data to be written into

In [11]:
queryFile = open("datapopulation1.txt", "w")

DETERMINE HOW MANY FALSE VALUES WE WANT FOR EACH

In [12]:
store_am = 6
warehouse_am = 5
products_am = 10
order_am = 20
num_customers = 20
card_am = 15 # total of card + giftcard ams need to be >= all customers bc every payment needs a corresponding payment
giftcard_am = 10 


DEFINE WHAT THE TABLES ARE CALLED IN SQL

In [13]:
warehouse = "warehouse"

order = "orders"

customerall = "customer"
customermember = "members"
customernonM = "nonMember"
customernonMonline = "onlineNM"

paymenttype = "paymentType"
creditinfo = "creditInfo"
giftcardinfo = "giftCardInfo"

store = "store"
onlineStore = "onlineStore"
physicalStore = "physicalStore"

Saving the inserted id's

In [14]:
store_ids = []
warehouse_ids = []
order_ids = []
customer_ids = []
custids_wpayment = []
payment_ids = []
product_ids = []

set starting values for id's 

In [15]:
store_id = 1
warehouse_id = 1
order_id = 1
customer_id = 1
member_id = 1
payment_id = 1
product_id = 1

generate: store data 

In [16]:
# Generating data for the singular online store- place in online store table and overall store table
# There is only one store_id for online store because there is only one online storefront
queryFile.write(f"insert into {store} values ('{store_id}');\n")
queryFile.write(f"insert into {onlineStore} values ('{store_id}', 'zara.com', '855.635.9272');\n")
store_ids.append(store_id)
store_id += 1

In [17]:
print(store_ids)

[1]


In [18]:
# Generating data for the physical stores- place in overall store and physical store tables
for i in range(store_am):
    # Calling the function we generated above
    generateS = generatePhysicalStore(store_id)
    values = ", ".join(f"'{str(v)}'" for v in generateS.values())
    # As the store_id increments, a new tuple is added to the physicalStore table and store table because of the hierarchy
    queryFile.write(f"insert into {store} values ('{store_id}');\n")
    queryFile.write(f"insert into {physicalStore} values ({values});\n")
    store_ids.append(store_id)
    store_id += 1

queryFile.write("\n")

1

In [19]:
print(store_ids)

[1, 2, 3, 4, 5, 6, 7]


generating warehouse data

In [20]:
# Writing a query to populate values into the warehouse entity 
for i in range(warehouse_am):
    # Calling the function we created above
    generateW = generateWarehouse(warehouse_id)
    values = ", ".join(f"'{str(v)}'" for v in generateW.values())
    queryFile.write(f"insert into {warehouse} values ({values});\n")
    warehouse_ids.append(warehouse_id)
    warehouse_id += 1

queryFile.write("\n")

1

In [21]:
print(warehouse_ids)

[1, 2, 3, 4, 5]


generate: orders data

In [22]:
# Writing a query to populate values into the order entity 
for i in range(order_am):
    generateO = generateOrder(order_id)
    values = ", ".join(f"'{str(v)}'" for v in generateO.values())
    queryFile.write(f"insert into {order} values ({values});\n")
    order_ids.append(order_id)
    order_id += 1

In [23]:
print(order_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [24]:
# Writing queries to populate values into the customers hierarchy
for cust in range(num_customers):
    # Randomly choosing whether the customer is a member, a non-member who purchased online, or a non-member who purchased
    # in the brick and mortar store
    customer_type = random.choice(["member", "nonmember_physical", "nonmember_online"])
    # Writing the query to insert the tuple into the customer entity because of the hierarchy, all customer_ids will be 
    # in the customer entity
    queryFile.write(f"insert into {customerall} values ('{customer_id}');\n")
    customer_ids.append(customer_id)

    if customer_type == "member":
        # If the customer is a member, generating their incrementing customer_id and member_id
        generateCM = generateCustomerMember(customer_id, member_id)
        values = ", ".join(f"'{str(v)}'" for v in generateCM.values())
        # Adding their member information to the members entity to be saved on their account
        queryFile.write(f"insert into {customermember} values ({values});\n")
        custids_wpayment.append(customer_id)
        member_id += 1
        
    elif customer_type == "nonmember_physical":
        generateNMP = generateCustomerNonMember(customer_id)
        values = ", ".join(f"'{str(v)}'" for v in generateNMP.values())
        queryFile.write(f"insert into {customernonM} values ({values});\n")
        
    elif customer_type == "nonmember_online":
        generateNMO = generateCustomerNonMembOnline(customer_id)
        values = ", ".join(f"'{str(v)}'" for v in generateNMO.values())
        queryFile.write(f"insert into {customernonM} values ('{customer_id}');\n")
        queryFile.write(f"insert into {customernonMonline} values ({values});\n")
        custids_wpayment.append(customer_id)

    customer_id += 1

queryFile.write("\n")

1

In [25]:
print(customer_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


generate: credit card info

In [26]:
# Writing queries to populate values into the credit card entity
for i in range(card_am):
    generateC = generateCardInfo(payment_id)
    values = ", ".join(f"'{str(v)}'" for v in generateC.values())
    
    queryFile.write(f"insert into {paymenttype} values ('{payment_id}');\n")
    queryFile.write(f"insert into {creditinfo} values ({values});\n")
    payment_ids.append(payment_id)

    payment_id += 1

queryFile.write("\n")

1

generate: gift card info

In [27]:
# Writing queries to populate values into the gift card entity
for i in range(giftcard_am):
    generateGC = generateGiftCardInfo(payment_id)
    values = ", ".join(f"'{str(v)}'" for v in generateGC.values())
    
    queryFile.write(f"insert into {paymenttype} values ('{payment_id}');\n")
    queryFile.write(f"insert into {giftcardinfo} values ({values});\n")
    payment_ids.append(payment_id)
    payment_id += 1

queryFile.write("\n")

1

Generating product hierarchy

Number of products we want to generate:

In [28]:
# Writing queries to populate values into the products hierarchy

for prod in range(products_am):
    product_category = random.choice(['womens', 'mens', 'babys', 'girls', 'boys'])

    if product_category in ['womens', 'mens', 'girls', 'boys']:
        main_type = random.choice(['Clothing', 'Shoes', 'Accessories'])
        if main_type == 'Clothing':
            clothing_type = random.choice(
                ['Outerwear', 'Sets', 'Tops', 'Bottoms', 'Swimwear', 'Suits', 'DressesAndJumpsuits'])
            if clothing_type == 'Outerwear':
                product_subcategory = 'Outerwear'
            elif clothing_type == 'Sets':
                product_subcategory = 'Set'
            elif clothing_type == 'Tops':
                product_subcategory = 'Top'
            elif clothing_type == 'Bottoms':
                product_subcategory = 'Bottom'
            elif clothing_type == 'Swimwear':
                product_subcategory = 'Swimwear'
            elif clothing_type == 'Suits':
                product_subcategory = 'Suit'
            elif clothing_type == 'DressesAndJumpsuits':
                product_subcategory = 'Dress/Jumpsuit'
        elif main_type == 'Shoes':
            product_subcategory = 'Shoes'
        elif main_type == 'Accessories':
            product_subcategory = 'Accessory'
    else:  # For babys
        main_type = random.choice(['Clothing', 'Accessories'])
        if main_type == 'Clothing':
            clothing_type = random.choice(['Outerwear', 'Sets', 'Tops', 'Bottoms'])
            if clothing_type == 'Outerwear':
                product_subcategory = 'Outerwear'
            elif clothing_type == 'Sets':
                product_subcategory = 'Set'
            elif clothing_type == 'Tops':
                product_subcategory = 'Top'
            elif clothing_type == 'Bottoms':
                product_subcategory = 'Bottom'
        elif main_type == 'Accessories':
            product_subcategory = 'Accessory'

    product_name = f"{product_category.capitalize()} {product_subcategory.capitalize()}"
    price = round(random.randint(20, 200))
    queryFile.write(f"insert into Products values ('{product_id}', '{product_name}', '{price}', '{product_category}');\n")

    product_ids.append(str(product_id))

    # Randomly deciding if it's clothing, shoes, or accessories
    main_type = random.choice(['Clothing', 'Shoes', 'Accessories'])

    if main_type == 'Clothing':
        clothing_color = random.choice(['red', 'blue', 'black', 'white', 'green'])
        queryFile.write(f"insert into Clothing values ('{product_id}', '{clothing_color}');\n")

        # Deciding clothing subcategory
        clothing_type = random.choice(['Outerwear', 'Sets', 'Tops', 'Bottoms', 'Swimwear', 'Suits', 'dressesAndJumpsuits'])

        if clothing_type == 'Outerwear':
            outerwear_material = random.choice(['Linen', 'Wool', 'Denim'])
            outerwear_size = random.choice(['XS', 'S', 'M', 'L', 'XL'])
            queryFile.write(f"insert into Outerwear values ('{product_id}', '{outerwear_material}', '{outerwear_size}');\n")

        elif clothing_type == 'Sets':
            sets_size = random.choice(['S', 'M', 'L', 'XL'])
            queryFile.write(f"insert into Sets values ('{product_id}', '{sets_size}');\n")

        elif clothing_type == 'Tops':
            top_type = random.choice(['T-shirt', 'Blouse', 'Sweater'])
            top_size = random.choice(['XS', 'S', 'M', 'L', 'XL'])
            queryFile.write(f"insert into Tops values ('{product_id}', '{top_type}', '{top_size}');\n")

        elif clothing_type == 'Bottoms':
            bottoms_material = random.choice(['Denim', 'Cotton', 'Polyester'])
            queryFile.write(f"insert into Bottoms values ('{product_id}', '{bottoms_material}');\n")

            # Deciding bottom subcategory
            bottom_type = random.choice(['Shorts', 'Pants', 'Skirts'])

            if bottom_type == 'Shorts':
                shorts_size = random.choice(['S', 'M', 'L'])
                shorts_type = random.choice(['Casual', 'Formal'])
                queryFile.write(f"insert into Shorts values ('{product_id}', '{shorts_size}', '{shorts_type}');\n")
            elif bottom_type == 'Pants':
                pants_size = random.choice(['30', '32', '34'])
                pants_length = random.choice(['Regular', 'Long'])
                pants_type = random.choice(['Jeans', 'Sweatpants'])
                queryFile.write(f"insert into Pants values ('{product_id}', '{pants_size}', '{pants_length}', '{pants_type}');\n")
            elif bottom_type == 'Skirts':
                skirt_size = random.choice(['S', 'M', 'L'])
                skirt_length = random.choice(['Mini', 'Midi', 'Maxi'])
                skirt_type = random.choice(['Pleated', 'Pencil'])
                queryFile.write(f"insert into Skirts values ('{product_id}', '{skirt_size}', '{skirt_length}', '{skirt_type}');\n")

        elif clothing_type == 'Swimwear':
            swimwear_type = random.choice(['One-piece', 'Bikini'])
            queryFile.write(f"insert into Swimwear values ('{product_id}', '{swimwear_type}');\n")

        elif clothing_type == 'Suits':
            suit_material = random.choice(['Wool', 'Linen'])
            suit_size = random.choice(['38R', '40R', '42R'])
            queryFile.write(f"insert into Suits values ('{product_id}', '{suit_material}', '{suit_size}');\n")

        elif clothing_type == 'DressesAndJumpsuits':
            dj_type = random.choice(['Dress', 'Jumpsuit'])
            neckline = random.choice(['V-neck', 'Round', 'Square'])
            sleeve_length = random.choice(['Short', 'Long', 'Sleeveless'])
            material = random.choice(['Cotton', 'Silk'])
            queryFile.write(f"insert into DressesAndJumpsuits values ('{product_id}', '{dj_type}', '{neckline}', '{sleeve_length}', '{material}');\n")

    elif main_type == 'Shoes':
        shoe_color = random.choice(['Black', 'White', 'Brown', 'Beige'])
        shoe_type = random.choice(['Sneakers', 'Boots', 'Sandals'])
        shoe_size = random.choice(['6', '7', '8', '9', '10'])
        queryFile.write(f"insert into Shoes values ('{product_id}', '{shoe_color}', '{shoe_type}', '{shoe_size}');\n")

    elif main_type == 'Accessories':
        queryFile.write(f"insert into Accessories values ('{product_id}');\n")

        accessory_type = random.choice(['Jewelry', 'Wearable', 'Bags', 'Hats'])

        if accessory_type == 'Jewelry':
            jewelry_material = random.choice(['Gold', 'Silver'])
            jewelry_type = random.choice(['Ring', 'Necklace'])
            jewelry_size = random.choice(['Small', 'Medium'])
            queryFile.write(f"insert into Jewelry values ('{product_id}', '{jewelry_material}', '{jewelry_type}', '{jewelry_size}');\n")

        elif accessory_type == 'Wearable':
            wearable_color = random.choice(['Black', 'Red'])
            wearable_size = random.choice(['Small', 'Medium'])
            wearable_type = random.choice(['Watch', 'Belt'])
            queryFile.write(f"insert into Wearable values ('{product_id}', '{wearable_color}', '{wearable_size}', '{wearable_type}');\n")

        elif accessory_type == 'Bags':
            bag_color = random.choice(['Black', 'Beige'])
            bag_type = random.choice(['Tote', 'Backpack'])
            queryFile.write(f"insert into Bags values ('{product_id}', '{bag_color}', '{bag_type}');\n")

        elif accessory_type == 'Hats':
            hat_color = random.choice(['Blue', 'White'])
            hat_size = random.choice(['S', 'M', 'L'])
            queryFile.write(f"insert into Hats values ('{product_id}', '{hat_color}', '{hat_size}');\n")


    product_id += 1
    
queryFile.write("\n")

1

In [29]:
print(product_ids)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


Generating the relationship data

In [30]:
# Writing queries to populate the customerOrder relation
customerOrder = "customerOrder"

for oid in order_ids:
    cust_id = random.choice(customer_ids)
    queryFile.write(f"insert into {customerOrder} values ('{cust_id}', '{oid}');\n")

In [31]:
# Writing queries to populate the customerPayment relation
customerPayment = "customerPayment"

pids_copy = payment_ids.copy()
cids_copy = custids_wpayment.copy() # this list is only the members that haev corresponding payments (online NMs and members)

# randomly assign one unique payment to each customer so that every customer is accounted for
for cust_id in cids_copy:
    pay_id = random.choice(pids_copy)
    queryFile.write(f"insert into {customerPayment} (customer_id, payment_id) values ('{cust_id}', '{pay_id}');\n")
    pids_copy.remove(pay_id)  # remove payment after assignment

# randomly assign leftover payment ids to random customers from all customers that have corresponding payment ids 
# (customers that are members + online non-members )
while pids_copy:
    pay_id = pids_copy.pop()
    cust_id = random.choice(custids_wpayment)  # pick from full customer list
    queryFile.write(f"insert into {customerPayment} (customer_id, payment_id) values ('{cust_id}', '{pay_id}');\n")

In [32]:
print(custids_wpayment)

[2, 3, 7, 8, 15, 17, 18]


In [33]:
# Writing queries to populate the storeOrderedFrom relation
storeOrderedFrom = "storeOrderedFrom"
for order_id in order_ids:
    chosen_store_id = random.choice(store_ids)
    queryFile.write(f"insert into {storeOrderedFrom} values ('{order_id}', '{chosen_store_id}');\n")

queryFile.write("\n")

1

In [34]:
# Writing queries to populate the itemsPurchased relation

itemsPurchased = "ItemsPurchased"

# Identifying the minimum and maximum number of items that can be in an order
min_items_per_order = 1
max_items_per_order = 50
for order_id in order_ids:
    # Defining an empty list of the products in the order
    products_in_order = []
    num_items = random.randint(min_items_per_order, max_items_per_order)
    for items in range(num_items):
        if product_ids:
            product_id = random.choice(product_ids)
            # Checking to make sure the product_id is not already in the list of products to avoid duplication
            if product_id not in products_in_order:
                price_at_purchase = round(random.randint(20,150))
                return_status_options = [None, "returned", "exchanged"]
                return_status = random.choice(return_status_options)
                date_of_return_sql = "NULL"
                if return_status == "returned" or return_status == "exchanged":
                    start_date = datetime.date.today() - datetime.timedelta(days=90)
                    end_date = datetime.date.today() - datetime.timedelta(days=7)
                    date_of_return = fake.date_between(start_date=start_date, end_date=end_date)
                    date_of_return_sql = f"'{date_of_return}'"
                
                quantity = random.randint(1, 10)
                queryFile.write(f"insert into {itemsPurchased} values ('{order_id}', '{product_id}','{price_at_purchase}','{quantity}', '{return_status}', {date_of_return_sql});\n") #added quantity
                products_in_order.append(product_id)

In [35]:
print(warehouse_ids)

[1, 2, 3, 4, 5]


In [36]:
# Writing queries to populate the supplies relation

Supplies = "Supplies"

for store in store_ids:
    warehouse = random.choice(warehouse_ids)
    queryFile.write(f"insert into {Supplies} (warehouse_id, store_id) values ('{warehouse}', '{store}');\n")

In [37]:
# Writing queries to populate the warehouseInventory relation

warehouseInventory = "warehouseInventory"

for wid in warehouse_ids:
    # Defining an empty list of products in the warehouse inventory
    products_in_warehouse_inventory = []
    for prod_id in product_ids:
        # Checking to make sure the product_id is not already in the list of products to avoid duplication
        if prod_id not in products_in_warehouse_inventory:
            quantity = random.randint(1, 200000)
            queryFile.write(f"insert into {warehouseInventory} values ('{wid}', '{prod_id}', '{quantity}');\n")
            products_in_warehouse_inventory.append(prod_id)

In [38]:
# Writing queries to populate the storeInventory relation
storeInventory = "storeInventory"

for store_id in store_ids:
    # Defining an empty list of products in the store inventory
    products_in_store_inventory = []
    for prod in range(random.randint(0, 1000)):
        product_id = random.choice(product_ids)
        # Checking to make sure the product_id is not already in the list of products to avoid duplication
        if product_id not in products_in_store_inventory:
            quantity = random.randint(0,1000)
            salepercent = random.uniform(0, 0.30)
            queryFile.write(f"insert into {storeInventory} values ('{store_id}', '{product_id}', '{quantity}', '{salepercent}');\n")
            products_in_store_inventory.append(product_id)

In [39]:
import mysql.connector

In [40]:
# Code we used to find where the errors were in our code
# Also how we read the text file we have been appending to and executed the queries one by one
# continuation from after commiting everything above 

try:
    # Making the connection to MySQL
    cnx = mysql.connector.connect(
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'), 
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'))
    cursor = cnx.cursor()
    
    # Reading the text file that we have been appending to
    with open('datapopulation1.txt', 'r') as dataFile:
        dataContent = dataFile.read()
        # Splitting on the semicolon and stripping for extra space
        queries = [q.strip() for q in dataContent.split(';') if q.strip()]

        # Executing the query
        for query in queries:
            print(f"Executing query: {query}")
            cursor.execute(query)

        # Comitting
        cnx.commit()
        print("Data population successful!")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    if 'cnx' in locals() and cnx.is_connected():
        # Rolling back the query if there was an error
        cnx.rollback()
        print("Transaction rolled back.")

# Closing the cursor and connection
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

Executing query: insert into store values ('1')
Error: 1062 (23000): Duplicate entry '1' for key 'store.PRIMARY'
Transaction rolled back.


In [41]:
# Closing the file
queryFile.close()

In [42]:
# open file with data that is dependant on other data 
queryFile = open("datapopulation2.txt", "w")

making tables that need data already loaded to create them

In [43]:
# Writing queries to populate the orderPayment relation
orderPayment = "orderPayment"

import mysql.connector  # or your database library

cnx = mysql.connector.connect(
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'), 
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'))
cursor = cnx.cursor()
    
for oid in order_ids:
    # find customer associated with the order
    cursor.execute(f"SELECT customer_id FROM customerOrder WHERE order_id = '{oid}';")
    customer_result = cursor.fetchone()
    
    if not customer_result:
        continue  # skip this order if no customer assigned
        
    customer_id = customer_result[0] # pulls result out of the tuple
    
    # find all payment IDs associated with this customer if the customer has an associated payment
    if customer_id in custids_wpayment:
        cursor.execute(f"SELECT payment_id FROM customerPayment WHERE customer_id = '{customer_id}';")
        payment_results = cursor.fetchall()
    
    else:
        print(f"Customer {customer_id} is non-member ordering from physical store - no payment data")
        continue  # skip if customer has no payment
    
    # pick a payment ID fromt the options (could just be one)
    customer_pids = [row[0] for row in payment_results]
    if customer_pids:
        pay_id = random.choice(customer_pids)
    else:
        print(f"no potential payment ids for customer {customer_id}")
        continue
    
    queryFile.write(f"insert into {orderPayment} (order_id, payment_id) values ('{oid}', '{pay_id}');\n")

Customer 19 is non-member ordering from physical store - no payment data
no potential payment ids for customer 17
Customer 14 is non-member ordering from physical store - no payment data
Customer 1 is non-member ordering from physical store - no payment data
Customer 14 is non-member ordering from physical store - no payment data
Customer 14 is non-member ordering from physical store - no payment data
Customer 14 is non-member ordering from physical store - no payment data
Customer 4 is non-member ordering from physical store - no payment data
Customer 10 is non-member ordering from physical store - no payment data
Customer 19 is non-member ordering from physical store - no payment data
Customer 20 is non-member ordering from physical store - no payment data
Customer 16 is non-member ordering from physical store - no payment data
Customer 16 is non-member ordering from physical store - no payment data
Customer 6 is non-member ordering from physical store - no payment data
Customer 20 i

In [44]:
# Code we used to find where the errors were in our code
# Also how we read the text file we have been appending to and executed the queries one by one
# continuation from after commiting everything above 

try:
    # Making the connection to MySQL
    cnx = mysql.connector.connect(
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'), 
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'))
    cursor = cnx.cursor()
    
    # Reading the text file that we have been appending to
    with open('datapopulation2.txt', 'r') as dataFile:
        dataContent = dataFile.read()
        # Splitting on the semicolon and stripping for extra space
        queries = [q.strip() for q in dataContent.split(';') if q.strip()]

        # Executing the query
        for query in queries:
            print(f"Executing query: {query}")
            cursor.execute(query)

        # Comitting
        cnx.commit()
        print("Data population successful!")

except mysql.connector.Error as err:
    print(f"Error: {err}")
    if 'cnx' in locals() and cnx.is_connected():
        # Rolling back the query if there was an error
        cnx.rollback()
        print("Transaction rolled back.")

# Closing the cursor and connection
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

Data population successful!


In [45]:
# Closing the file
queryFile.close()